In [328]:
import numpy as np
import pandas as pd
import numba

NMF with numba

In [2]:
def WKLNMF(N, k, eps, maxit):
    P = N / N.sum(axis = 0)
    V, D = N.shape
    W = N
    B = np.random.uniform(size = (V, k))
    B = B / B.sum(axis = 0)

    Theta = np.random.uniform(size = (k, D))
    Theta = Theta/Theta.sum(axis = 0)
    
    for i in range(maxit):
        Theta = np.multiply(np.divide(Theta, np.matmul(B.transpose(),W)), 
                            np.matmul(B.transpose(),np.divide(np.multiply(W,P),np.matmul(B,Theta))))
        B = np.multiply(np.divide(B, np.matmul(W, Theta.transpose())), 
                        np.matmul(np.divide(np.multiply(W, P), np.matmul(B, Theta)), Theta.transpose()))
        #print(KL(W,P,B,Theta))
        if KL(W, P, B, Theta) < eps:
            break
    #print(i)
    
    return B, Theta

def KL(W,P,B,Theta):
    loss = 0
    P_hat = np.matmul(B, Theta)
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            loss += W[i,j]*(P[i,j]*np.log(P[i,j]/P_hat[i,j])-P[i,j] + P_hat[i,j])
            
    return loss



In [15]:
N = np.random.randint(1,20,size = (10000,20000))
N = N / N.sum(axis = 0)
k = 20
eps = 10e-2
maxit = 100

In [5]:
%%time 
WKLNMF(N, k, eps, maxit)

C:\Users\barry\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in log
C:\Users\barry\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars


Wall time: 22.3 s


(array([[0.00746431, 0.00055929, 0.00920337, ..., 0.00110429, 0.02092352,
         0.00153673],
        [0.00539106, 0.00083921, 0.00384299, ..., 0.0007203 , 0.00280014,
         0.01123968],
        [0.00567421, 0.01718279, 0.00100391, ..., 0.00164398, 0.00456805,
         0.00995607],
        ...,
        [0.00586525, 0.00432757, 0.00288669, ..., 0.00449483, 0.00464201,
         0.02220088],
        [0.0164468 , 0.02154836, 0.01036647, ..., 0.00111193, 0.00251421,
         0.01047645],
        [0.00074129, 0.00960939, 0.00406671, ..., 0.00596256, 0.00304758,
         0.01688359]]),
 array([[0.15933048, 0.12496188, 0.0985239 , ..., 0.06216874, 0.18852865,
         0.09481424],
        [0.00277351, 0.10557129, 0.01432642, ..., 0.00749124, 0.00644578,
         0.01232335],
        [0.03982357, 0.0388621 , 0.15884498, ..., 0.15941586, 0.00697769,
         0.07341061],
        ...,
        [0.0012296 , 0.09773317, 0.00698148, ..., 0.01204276, 0.08616812,
         0.06580205],
        [0.1

In [13]:
@numba.jit
def WKLNMF(P, k, eps, maxit):
    V, D = P.shape
    W = P
    p_max = P.max()
    p_min = P.min()

    B = np.random.uniform(p_min,p_max,size = (V, k))
    B = B / B.sum(axis = 0)

    Theta = np.random.uniform(p_min,p_max,size = (k, D))
    Theta = Theta/Theta.sum(axis = 0)
    
    for i in numba.prange(maxit):
        Theta = (Theta/np.matmul(B.T,W))*np.matmul(B.T,(W*P)/np.matmul(B,Theta))
        B = (B/np.matmul(W, Theta.T))*np.matmul((W*P)/np.matmul(B, Theta), Theta.T)
        KL_div = KL(W,P,B,Theta)
        print(KL_div)
        #print(KL(W,P,B,Theta))
        if KL_div < eps:
            break
    #print(i)
    
    return B, Theta
@numba.jit
def KL(W,P,B,Theta):
    loss = 0
    P_hat = np.matmul(B, Theta)
    
    loss = np.sum(W*(P*np.log(P/P_hat) - P + P_hat))
            
    return loss



In [16]:
%%time
b, theta = WKLNMF(N, k, eps, maxit)

0.19098134424589527
0.19076987956297808
0.19057294499531388
0.19038870774153505
0.19021602607383162


KeyboardInterrupt: 

In [17]:
import os
td_matrix_pd = pd.read_csv(os.path.join(os.getcwd(),'term-document matrix','Matrix_interjection_tfidf.csv'))
td_matrix_pd = td_matrix_pd.replace(0,1e-10)
td_matrix = td_matrix_pd.values
td_matrix = td_matrix / td_matrix.sum(axis = 0)

In [18]:
td_matrix.max()
td_matrix.min()

7.070525165377817e-18

In [23]:
%%time
b, theta = WKLNMF(N, 40, eps, maxit)

0.18859092234786531
0.1885283474672066
0.1884681370666125
0.18840992189552155
0.18835360659962055
0.18829910175178768
0.18824632339854752
0.1881951926476393
0.18814563529240613
0.18809758146916272
0.18805096534429308
0.1880057248280113
0.18796180131225806
0.18791913943029956
0.18787768683602354
0.1878373940010463
0.18779821402796362
0.18776010247827069
0.18772301721364462
0.18768691824934955
0.1876517676187425
0.18761752924785943
0.18758416883927362
0.18755165376438412
0.18751995296345436
0.18748903685275795
0.18745887723823845
0.18742944723515567
0.1874007211932508
0.18737267462698354
0.18734528415043744
0.18731852741655633
0.1872923830603417
0.18726683064577185
0.18724185061608078
0.18721742424722462
0.18719353360425908
0.18717016150043492
0.18714729145880077
0.18712490767614348
0.1871029949891209
0.18708153884238507
0.1870605252586214
0.1870399408103089
0.18701977259314637
0.18700000820097987
0.18698063570217394
0.1869616436172985
0.18694302089806866
0.1869247569074527
0.18690684140

NMF with ANLS

In [329]:
@numba.jit
def NMF_ANLS_full(P,k):

    B = np.random.uniform(low = 1, high = 1000,size = (P.shape[0], k))
    B = B/B.sum(axis = 0)

    return B, NMF_ANLS(B,P)

@numba.jit
def NMF_ANLS(B, P):
    
    theta = np.zeros(shape = (B.shape[1],P.shape[1]))
    
    for i,column in enumerate(P.T):
        theta[:,i] = ANLS_column(B, column)
        
    return theta

@numba.jit
def ANLS_column(B, y):
    
    m,n = B.shape
    g = np.zeros(n)
    E = np.arange(n)
    S = np.array([])
    w = np.matmul(B.T, y-np.matmul(B,g))
    

    while len(E) != 0 and w[E].max() > 0:
        
        t = w[E].argmax()
        E = np.delete(E, t).astype(int)
        S = np.append(S,t).astype(int)
        
        Bs = B.copy()
        Bs[:,E] = 0
        
        z = np.linalg.lstsq(Bs, y, rcond=None)[0]
        z[E] = 0

        while z[S].max() <= 0:
            
            alpha = (g/(g-z))[S].min()
            
            g = g + alpha*(z-g)
            
            s_move = S[g[S] == 0]
            S = np.delete(S, s_move).astype(int)
            E = np.append(E, s_move).astype(int)
            Bs = B.copy()
            Bs[:,E] = 0            
            z = np.linalg.lstsq(Bs, y, rcond=None)[0]
            #z[E] = 0
        g = z
        w = np.matmul(B.T, y-np.matmul(B,g))
    
    return g

In [319]:
P = np.random.randint(low = 1, high = 100, size = (50,100))
P = P/P.sum(axis = 0)

In [332]:
import pandas as pd
import os
#td_matrix_pd = pd.read_csv(os.path.join(os.getcwd(),'term-document matrix','Matrix_interjection_tfidf.csv'))
td_matrix_pd = td_matrix_pd.replace(0,1e-10)
td_matrix = td_matrix_pd.values
td_matrix = td_matrix / td_matrix.sum(axis = 0)

In [337]:
%%time
B, theta = NMF_ANLS_full(td_matrix, 40)

Wall time: 19min 11s


In [336]:
B_first = B
theta_first = theta

In [340]:
B

array([[2.94712772e-04, 6.95464179e-05, 9.44301738e-05, ...,
        3.84266404e-05, 2.43194934e-04, 2.39614257e-04],
       [3.40411725e-04, 2.08899072e-04, 6.54339596e-05, ...,
        2.27072622e-04, 1.90630476e-04, 3.03354191e-04],
       [2.17207526e-04, 2.43072041e-04, 1.67023961e-04, ...,
        6.29972748e-06, 1.57955835e-04, 1.50210535e-04],
       ...,
       [3.04181734e-04, 1.89596396e-04, 1.75017323e-04, ...,
        2.98197940e-04, 6.12850206e-05, 1.95914146e-04],
       [4.46340813e-05, 1.07273925e-04, 1.38943421e-05, ...,
        1.00181831e-04, 3.05002402e-04, 3.32230019e-04],
       [2.92562957e-04, 3.69910190e-05, 1.44337704e-04, ...,
        6.95121413e-05, 1.98232899e-04, 3.27610491e-04]])